In [ ]:
# 這個 Python 3 環境安裝了許多有用的分析庫
# 它由 kaggle/python Docker 鏡像定義：https://github.com/kaggle/docker-python
# 例如，這裡有幾個有用的包可以加載例如

import numpy as np # 線性代數 #建立向量（Vector）、矩陣（Matrix）等進行高效率的大量資料運算
import pandas as pd # 數據處理、CSV 文件 I/O（例如 pd.read_csv） #進行資料處理和資料分析
import glob #文件搜索

from joblib import Parallel, delayed #用於併行處理

import xgboost as xgb
from xgboost.sklearn import XGBRegressor


import os #對檔案進行重新命名、刪除等一系列操作
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#os.path.join=>合並目錄

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()  
    #計算對數回報，取兩個連續的WAP之間的比率的對數
    #log=>提取自然對數 https://www.delftstack.com/zh-tw/api/numpy/python-numpy-log/
    #diff=>計算向量的連續元素之間的差值 https://www.delftstack.com/zh-tw/howto/r/diff-function-in-r/

def realized_volatility(series_log_return): #已實現波動率
    return np.sqrt(np.sum(series_log_return**2)) #sqrt=>平方 sum=>加總

In [ ]:
book_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
book_test = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0')
trade_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
trade_test = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0')


book_example = book_train[book_train["time_id"]==5]
book_example

#stock_id- 股票的 ID 代碼。並非所有股票 ID 都存在於每個時間段中。Parquet 在加載時會將此列強制為分類數據類型；您可能希望將其轉換為 int8。
#time_id- 時間段的 ID 代碼。時間 ID 不一定是連續的，但在所有股票中都是一致的。
#seconds_in_bucket- 從桶開始的秒數，總是從 0 開始。
#bid_price[1/2]- 最具/第二最具競爭力的購買水平的標準化價格。
#ask_price[1/2]- 最具/第二最具競爭力的銷售水平的標準化價格。
#bid_size[1/2]- 最具/第二最具競爭力的買入水平的股票數量。
#ask_size[1/2]- 最具/第二最具競爭力的賣出水平的股票數量。

In [ ]:
trade_example = trade_train[trade_train["time_id"]==5]
trade_example

#stock_id - 同上。
#time_id - 同上。
#seconds_in_bucket - 同上。請注意，由於交易和賬面數據取自同一時間窗口，並且交易數據通常更稀疏，因此該字段不一定從 0 開始。
#price - 在一秒鐘內發生的已執行交易的平均價格。價格已標準化，平均價格已按每筆交易中交易的股票數量加權。
#size - 交易的股票總數。
#order_count - 發生的唯一交易訂單的數量。
#train.csv 訓練集的真實值。

In [ ]:
book_test

#stock_id - 同上。
#time_id - 同上。
#seconds_in_bucket - 同上。請注意，由於交易和賬面數據取自同一時間窗口，並且交易數據通常更稀疏，因此該字段不一定從 0 開始。
#price - 在一秒鐘內發生的已執行交易的平均價格。價格已標準化，平均價格已按每筆交易中交易的股票數量加權。
#size - 交易的股票總數。
#order_count - 發生的唯一交易訂單的數量。
#train.csv 訓練集的真實值。

In [ ]:
trade_test

#stock_id - 同上。
#time_id - 同上。
#seconds_in_bucket - 同上。請注意，由於交易和賬面數據取自同一時間窗口，並且交易數據通常更稀疏，因此該字段不一定從 0 開始。
#price - 在一秒鐘內發生的已執行交易的平均價格。價格已標準化，平均價格已按每筆交易中交易的股票數量加權。
#size - 交易的股票總數。
#order_count - 發生的唯一交易訂單的數量。
#train.csv - 訓練集的真實值。

In [ ]:
def get_stock_stat(stock_id : int, dataType = 'train'):
    
    book_train_subset = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_{dataType}.parquet/stock_id={stock_id}/') #讀取資料
    book_train_subset.sort_values(by=['time_id', 'seconds_in_bucket'])  #sort_values=>排序 https://www.delftstack.com/zh-tw/api/python-pandas/pandas-dataframe-dataframe.sort_values-function/

    book_train_subset['bas'] = (book_train_subset[['ask_price1', 'ask_price2']].min(axis = 1)#每一行的最小值 axis=1=>行 axis=0=>列 最佳報價
                                / book_train_subset[['bid_price1', 'bid_price2']].max(axis = 1)#每一行的最大值  最佳買入價
                                - 1)  #買賣差價                             

    
    book_train_subset['wap'] = (book_train_subset['bid_price1'] * book_train_subset['ask_size1'] +
                            book_train_subset['ask_price1'] * book_train_subset['bid_size1']) / (
                            book_train_subset['bid_size1']+ book_train_subset['ask_size1']) #計算對數收益率 加權平均價格

    book_train_subset['log_return'] = (book_train_subset.groupby(by = ['time_id'])['wap'].
                                       apply(log_return).
                                       reset_index(drop = True).
                                       fillna(0)
                                      ) #將wap依照time_id進行分組 groupby=>分割成多個組 https://www.delftstack.com/zh-tw/howto/python-pandas/pandas-groupby-count-python/
                                        #apply=>對每一行  https://www.delftstack.com/zh-tw/api/python-pandas/pandas-dataframe-dataframe.apply-function/
                                        #reset_index=(drop = True)>重置回預設索引 https://www.delftstack.com/zh-tw/api/python-pandas/pandas-dataframe-dataframe.reset_index-function/
                                        #fillna(0)=>用0取代空直 https://medium.com/@zector1030/pandas-fillna-%E7%AF%84%E4%BE%8B-5d33819fb7b8

    stock_stat = pd.merge(
        book_train_subset.groupby(by = ['time_id'])['log_return'].agg(realized_volatility).reset_index(), #agg獲取統計資料 https://www.delftstack.com/zh-tw/howto/python-pandas/pandas-groupby-count-python/   https://www.delftstack.com/zh-tw/howto/python-pandas/how-to-count-unique-values-with-pandas-per-groups/
        book_train_subset.groupby(by = ['time_id'], as_index = False)['bas'].mean(),
        on = ['time_id'],
        how = 'left'
    )
    #merge=>合併索引 https://www.delftstack.com/zh/api/python-pandas/pandas-dataframe-dataframe.merge-function/
    #mean=>平均值
    #on=>要合并的列或索引名称
    #how=>如何进行合并操作
    #as_index = False =>  index的標題位置上移 https://blog.csdn.net/cjsyr6wt/article/details/78200444
    stock_stat['stock_id'] = stock_id
    
    return stock_stat

In [ ]:
def get_dataSet(stock_ids : list, dataType = 'train'):

    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat)(stock_id, dataType) 
        for stock_id in stock_ids  #將stock_ids列表中指派到stock_id=>就是讓全部的stock_id都有跑到
        )
    """
    stock_stat=>股票統計
    Parallel=>平行運算
    n_jobs=-1=>表示cpu裡的所有core進行工作 https://medium.com/bryanyang0528/scikit-learn-%E4%B8%AD-kmeans%E7%9A%84n-job%E5%8F%83%E6%95%B8%E6%9C%83%E8%AE%93%E7%B5%90%E6%9E%9C%E4%B8%8D%E4%B8%80%E8%87%B4-7ce8762ac5f9
                                        https://zhuanlan.zhihu.com/p/180347090
    delayed=>创建元组 https://blog.csdn.net/Flag_ing/article/details/106972790
    """
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)
    #concat=>合併欄位
    #ignore_index = True=> 可以忽略合併時舊的 index 欄位，改採用自動產生的 index http://violin-tao.blogspot.com/2017/06/pandas-2-concat-merge.html
    return stock_stat_df

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')#讀取資料

train_stock_stat_df = get_dataSet(stock_ids = train['stock_id'].unique(), dataType = 'train') #unique()=>保留唯一值 https://www.delftstack.com/zh-tw/api/python-pandas/pandas-series-series.unique-function/
train_dataSet = pd.merge(train, train_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left') #merge=>合併索引 https://www.delftstack.com/zh/api/python-pandas/pandas-dataframe-dataframe.merge-function/

train_stock_stat_df

In [ ]:
train_dataSet

In [ ]:
# 數據最後準備
y_train = train_dataSet['target']
X_train = train_dataSet.drop(['stock_id', 'time_id', 'target'], axis = 1) #從train_dataSet刪除'stock_id', 'time_id', 'target'的列 #https://www.delftstack.com/zh-tw/howto/python-pandas/how-to-delete-pandas-dataframe-column/
X_train

In [ ]:
y_train

In [ ]:
clf = XGBRegressor(random_state = 0
                   #,n_estimators = 200
                   #,learning_rate = 0.1
                   #,subsample = 0.8
                   #,colsample_bytree = 0.8
                   ,n_jobs= - 1)
        #https://blog.csdn.net/m0_37870649/article/details/82707197
        #https://blog.csdn.net/SeaSky_Steven/article/details/105540245
        #
clf.fit(X_train,y_train.to_numpy().ravel()) 
        #to_numpy()=>將Dataframe轉換為NumPy陣列 https://www.delftstack.com/zh-tw/howto/python-pandas/how-to-convert-pandas-dataframe-to-numpy-array/
        #ravel()=>將矩陣轉換為NumPy中的陣列 https://www.delftstack.com/zh-tw/howto/numpy/matrix-to-array-numpy/

In [ ]:
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')#讀取資料

test

In [ ]:
test_stock_stat_df = get_dataSet(stock_ids = test['stock_id'].unique(), dataType = 'test') #unique()=>保留唯一值 https://www.delftstack.com/zh-tw/api/python-pandas/pandas-series-series.unique-function/
test_dataSet = pd.merge(test, test_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left') #merge=>合併索引 https://www.delftstack.com/zh/api/python-pandas/pandas-dataframe-dataframe.merge-function/
test_dataSet = test_dataSet.drop(['stock_id', 'time_id'], axis = 1)

test_dataSet

In [ ]:
y_pred = test_dataSet[['row_id']]
X_test = test_dataSet.drop(['row_id'], axis = 1).fillna(0) #fillna=>填充所有的 NaN 值
X_test

In [ ]:
y_pred1 = y_pred.assign(target = clf.predict(X_test)) #predict=>使用模型做預測 https://rstudio-pubs-static.s3.amazonaws.com/285162_5c3505e33774413bb39f243328ec97c8.html
y_pred1.to_csv('submission.csv',index = False) #https://blog.csdn.net/toshibahuai/article/details/79034829 #https://vimsky.com/zh-tw/examples/detail/python-ex-pandas-DataFrame-to_csv-method.html
y_pred1